# 5 - Predição do modelo final

Este notebook implementa a predição do modelo preditivo para estimar a **demanda horária** do pronto-socorro exportado nas etapas anteriores

Os principais objetivos deste notebook são:
- Fazer a predição de uma janela futura de dados;
- Validar os resultados
---

# 1. Importar bibliotecas e configurações iniciais

### 1.0 Instalações

In [22]:
# ! pip install scikit-learn

### 1.1 Importações

In [23]:
import pandas as pd
import numpy as np
import joblib
import altair as alt
import locale

In [24]:
import sys
sys.path.append("../src")

from features.feature_engineering import create_lag_features, create_rolling_features, add_time_features
from forecast.recursive_forecast import recursive_forecast

### 1.2 Configurações de bibliotecas

In [25]:
# Desabilitar a limitação de linhas em gráficos do Altair
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [26]:
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

# 2. Importar e tratar dados

### 2.1 Importando dados

In [27]:
CAMINHO_DADOS = '../data/raw/dataset_pronto_socorro.csv'

df = pd.read_csv(CAMINHO_DADOS)

In [28]:
df.head()

,datetime,day_of_week,month,is_weekend,temperature,rain_mm,demand
0,2023-01-01 00:00:00,6,1,1,24.483571,0.353269,29.0
1,2023-01-01 01:00:00,6,1,1,21.308678,5.847757,30.0
2,2023-01-01 02:00:00,6,1,1,25.238443,1.141991,30.0
3,2023-01-01 03:00:00,6,1,1,29.615149,0.524987,33.0
4,2023-01-01 04:00:00,6,1,1,20.829233,0.820610,33.0


### 2.2 Tratamento de Dados

In [29]:
df.drop(columns=['day_of_week'], inplace=True)

In [30]:
# Converter coluna de data/hora para datetime e definir índice
if 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.set_index('datetime')

### 2.3 Engenharia de Featurees

In [31]:
df = df.copy()

# Adicionar variáveis temporais
df = add_time_features(df)

# Lags essenciais: 1 hora, 24h, 48h, 1 semana (168h)
df = create_lag_features(df, lags=[1, 2, 3, 24, 48, 168])

# Rolling windows
df = create_rolling_features(df, windows=[3, 6, 12, 24])

# Remover linhas com NaNs causados pelos lags/rolling
df = df.dropna()

In [32]:
df.head()

,month,is_weekend,temperature,rain_mm,demand,hour,dayofweek,demand_lag_1,demand_lag_2,demand_lag_3,...,demand_lag_48,demand_lag_168,demand_roll_mean_3,demand_roll_std_3,demand_roll_mean_6,demand_roll_std_6,demand_roll_mean_12,demand_roll_std_12,demand_roll_mean_24,demand_roll_std_24
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-08 00:00:00,1,1,20.773059,2.344874,25.0,0,6,28.0,26.0,20.0,...,22.0,29.0,26.333333,1.527525,24.166667,3.060501,24.083333,3.704011,25.583333,3.877658
2023-01-08 01:00:00,1,1,18.231319,0.268283,25.0,1,6,25.0,28.0,26.0,...,27.0,30.0,26.000000,1.732051,24.833333,2.639444,23.833333,3.511885,25.375000,3.762660
2023-01-08 02:00:00,1,1,17.552428,0.192845,22.0,2,6,25.0,25.0,28.0,...,24.0,30.0,24.000000,1.732051,24.333333,2.875181,23.583333,3.528026,25.000000,3.623594
2023-01-08 03:00:00,1,1,17.920949,1.191458,34.0,3,6,22.0,25.0,25.0,...,26.0,33.0,27.000000,6.244998,26.666667,4.082483,24.000000,4.410731,25.250000,4.024382
2023-01-08 04:00:00,1,1,21.614491,3.771095,35.0,4,6,34.0,22.0,25.0,...,23.0,33.0,30.333333,7.234178,28.166667,5.269409,24.833333,5.441145,25.666667,4.488310


In [33]:
# Opcional: features sazonais contínuas (útil até pro baseline)

# Sinais "sin" e "cos" transformam hora em um ciclo contínuo, evitando que "23" esteja longe de "0".

# df['sin_hour'] = np.sin(2 * np.pi * df['hour'] / 24)
# df['cos_hour'] = np.cos(2 * np.pi * df['hour'] / 24)

### 2.4 Carregar o modelo exportado

In [34]:
best_model = joblib.load("../models/model_final.pkl")

FEATURES = joblib.load("../models/features.pkl")

In [35]:
best_model

,steps,"[('scaler', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,None


# 3 Predição Futura

In [36]:
HORIZON = 24 * 7  # 7 dias de horizonte

In [37]:
FEATURES

['month',
 'is_weekend',
 'hour',
 'dayofweek',
 'demand_lag_1',
 'demand_lag_2',
 'demand_lag_3',
 'demand_lag_24',
 'demand_lag_48',
 'demand_lag_168',
 'demand_roll_mean_3',
 'demand_roll_std_3',
 'demand_roll_mean_6',
 'demand_roll_std_6',
 'demand_roll_mean_12',
 'demand_roll_std_12',
 'demand_roll_mean_24',
 'demand_roll_std_24']

In [38]:
future = recursive_forecast(df.tail(200), best_model, FEATURES, steps=6)

2025-11-24 19:28:09,410 - INFO - Prevendo para 2024-12-31 01:00:00
2025-11-24 19:28:09,414 - INFO - Previsão: 20.9960908848535
2025-11-24 19:28:09,416 - INFO - Prevendo para 2024-12-31 02:00:00
2025-11-24 19:28:09,419 - INFO - Previsão: 22.992649281912005
2025-11-24 19:28:09,420 - INFO - Prevendo para 2024-12-31 03:00:00
2025-11-24 19:28:09,425 - INFO - Previsão: 18.00574574154249
2025-11-24 19:28:09,426 - INFO - Prevendo para 2024-12-31 04:00:00
2025-11-24 19:28:09,429 - INFO - Previsão: 20.99735541257612
2025-11-24 19:28:09,430 - INFO - Prevendo para 2024-12-31 05:00:00
2025-11-24 19:28:09,433 - INFO - Previsão: 22.988536364151795
2025-11-24 19:28:09,434 - INFO - Prevendo para 2024-12-31 06:00:00
2025-11-24 19:28:09,438 - INFO - Previsão: 18.011808677446794


In [39]:
future['demand']

datetime
2024-12-31 01:00:00    20.996091
2024-12-31 02:00:00    22.992649
2024-12-31 03:00:00    18.005746
2024-12-31 04:00:00    20.997355
2024-12-31 05:00:00    22.988536
2024-12-31 06:00:00    18.011809
Name: demand, dtype: float64

# 3. Avaliação da Predição Futura

In [40]:
hist = df[["demand"]].tail(24*5).reset_index()
hist["type"] = "hist"

fut = future[["demand"]].reset_index()
fut["type"] = "forecast"

plot_df = pd.concat([hist, fut])

In [42]:
chart_compare = (
    alt.Chart(plot_df)
        .mark_line()
        .encode(
            x="datetime:T",
            y="demand:Q",
            color="type:N"
        )
        .properties(title="Histórico vs Previsão")
).properties(width=800, height=400)

chart_compare

alt.Chart(...)

# 4. Conclusão

Depois que é feita a primeira previsão futura, passamos a considerar:
- valores reais → até o último instante observado
- previsões → para todas as horas futuras

Isso significa que a cada iteração, os lags ficam cada vez mais baseados únicamente em previsões.

E previsões geradas por regressão linear:
- tendem para a média (mean reverting)
- têm pouca variabilidade
- convergem para um ponto fixo (“atrator”)

**O modelo linear está "colapsando" para uma solução estática, reforçada pelos próprios lags.**

**Problema importante:** o modelo está superdependente das janelas e dos lags

Se os lags forem quase idênticos (por exemplo 20 → 22 → 18 → 20…), o modelo linear não tem capacidade de gerar dinâmica real.

Por isso o ciclo de valores é consequência direta do uso dos próprios lags previstos como entradas.

- Este é um comportamento esperado de modelos lineares recursivos com lags.